In [29]:
import pandas as pd
import numpy as np
import json

from sklearn import preprocessing, feature_extraction, model_selection
from sklearn.metrics import mean_absolute_error, mean_squared_error

import stellargraph as sg
from stellargraph.mapper import HinSAGELinkGenerator
from stellargraph.layer import HinSAGE, link_regression
from tensorflow.keras import Model, optimizers, losses, metrics

import multiprocessing
from IPython.display import display, HTML
import matplotlib.pyplot as plt
%matplotlib inline

In [5]:
import sys
sys.path.append('../')

import run

v_sets, e_sets, core_targets, ext_targets = run.main()

Running the scripts sequentially 

LOADING DATA: 6.77 s
SUBSAMPLING: 0.05 s
PREPROCESSING: Complete in 0.04 s


In [23]:
# Initialize SG graph
G = sg.StellarDiGraph(v_sets, e_sets)

In [19]:
# CLASSIFICATION WITH GRAPHSAGE

# Split in train(70%), test(15%) and validation set (15%) 
train_ID, test_ID = train_test_split(
  ext_targets, train_size=0.7, test_size=None, #stratify=ext_targets
)
val_ID, test_ID = train_test_split(
  test_ID, train_size=0.5, test_size=None, #stratify=test_ID
)


# Convert targets labels in one-hot encoded features (optional, for categorical targets)
target_encoding = LabelBinarizer()
train_targets = target_encoding.fit_transform(train_ID)
val_targets = target_encoding.transform(val_ID)
test_targets = target_encoding.transform(test_ID)

In [42]:
train_targets.shape

(1902, 1177)

In [20]:
model_type = "graphsage" # gcn, gat
use_bagging = (True)

In [21]:
batch_size = 50
num_samples = [10, 10]
n_estimators = 5  # The number of estimators in the ensemble
n_predictions = 10  # The number of predictions per estimator per query point
epochs = 50  # The number of training epochs

In [24]:
generator = GraphSAGENodeGenerator(G, batch_size, num_samples)
train_gen = generator.flow(train_ID.index, train_targets, shuffle=True)

<ipython-input-24-78ecb2afb957>:1: RuntimeWarning: running homogeneous GraphSAGE on a graph with multiple node types
  generator = GraphSAGENodeGenerator(G, batch_size, num_samples)


ValueError: node_ids: expected all nodes to be of type Account, found some nodes with wrong type: 11576, 15800, 16573, 13269, 17866, 15447, 13750, 14890, 13579, 16975, 14735, 17237, 16514, 16352, 10938, 13573, 12284, 14020, 15604, 12258, ... (1026 more)

In [31]:
base_model = GraphSAGE(
    layer_sizes=[16, 16],
    generator=generator,
    bias=True,
    dropout=0.5,
    normalize="l2"
)

x_in, x_out = base_model.in_out_tensors()
predictions = layers.Dense(units=train_targets.shape[1], activation="softmax")(x_out)

RuntimeError: GraphSAGE called on graph with more than one node type.

In [32]:
model = Model(inputs=x_in, outputs=predictions)

if use_bagging:
    model = BaggingClassifier(model, n_estimators=n_estimators, n_predictions=n_predictions)

NameError: name 'x_in' is not defined

In [33]:
model.compile(
  optimizer=optimizers.Adam(lr=0.005),
  loss=losses.categorical_crossentropy,
  metrics=["acc"],
)

NameError: name 'model' is not defined

In [34]:
val_gen = generator.flow(val_ID.index, val_targets)
test_gen = generator.flow(test_ID.index, test_targets)

ValueError: node_ids: expected all nodes to be of type Account, found some nodes with wrong type: 18318, 16927, 13499, 16417, 14687, 18337, 11884, 12126, 13970, 11562, 11693, 11937, 13860, 11494, 13923, 16697, 18154, 11174, 11111, 12437, ... (200 more)

In [35]:
if use_bagging:
    # When using bootstrap samples to train each model in the ensemble, we must specify
    # the IDs of the training nodes (train_data) and their corresponding target values
    # (train_targets)
    history = model.fit(
      generator,
      train_data=train_gen.index,
      train_targets=train_targets,
      epochs=epochs,
      validation_data=val_gen,
      verbose=0,
      shuffle=False,
      bag_size=None,
      use_early_stopping=True,  # Enable early stopping
      early_stopping_monitor="val_acc",
    )

NameError: name 'model' is not defined

In [36]:
sg.utils.plot_history(history)

NameError: name 'history' is not defined

In [38]:
test_metrics = model.evaluate(test_gen)
print("\nTest Set Metrics:")
for name, val in zip(model.metrics_names, test_metrics):
    print("\t{}: {:0.4f}".format(name, val))

NameError: name 'model' is not defined